In [77]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import lightgbm as lgb

In [78]:
label = pd.read_csv('/Users/yangdongjae/Desktop/2020/대외활동/2020년 공공 빅데이터 청년 인턴십/실무형 프로젝트/roundabout_project/data/전국사고csv.csv', encoding = 'cp949')

In [79]:
label

,OBJT_ID,OCCU_YEAR,OCCU_MT,OCCU_DE,OCCU_TM,OCCU_DAY,OCCU_DATE,LCLAS,SCLAS,DEATH,...,DRNKG,KID,ODSN,WLKG,VIOLT_CN,CTPRVN_CD,SGG_CD,EMD_CD,X,Y
0,1,2016,2,27,16,토요일,20160227,경상사고,차대차,0,...,NaN,NaN,NaN,NaN,안전거리 미확보,11,11110,11110101,14134215.68,4521788.508
1,2,2016,2,28,19,일요일,20160228,중상사고,차대차,0,...,NaN,NaN,NaN,NaN,안전운전 의무 불이행,11,11110,11110101,14134196.19,4521218.594
2,3,2016,3,28,7,월요일,20160328,중상사고,차대차,0,...,NaN,NaN,NaN,NaN,안전거리 미확보,11,11110,11110101,14134551.99,4521176.288
3,4,2016,4,5,13,화요일,20160405,경상사고,차대차,0,...,NaN,NaN,O,NaN,안전운전 의무 불이행,11,11110,11110101,14134007.37,4521163.076
4,5,2016,5,15,19,일요일,20160515,경상사고,차대차,0,...,NaN,O,NaN,NaN,안전거리 미확보,11,11110,11110101,14134220.38,4521176.947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654354,654355,2018,12,15,16,토요일,20181215,경상사고,차대차,0,...,NaN,NaN,NaN,NaN,신호위반,50,50130,50130320,14118562.64,3939567.424
654355,654356,2018,12,15,23,토요일,20181215,경상사고,차대차,0,...,NaN,NaN,O,NaN,신호위반,50,50130,50130320,14118368.46,3940935.052
654356,654357,2018,12,18,18,화요일,20181218,중상사고,차대차,0,...,NaN,NaN,NaN,NaN,안전운전 의무 불이행,50,50130,50130320,14117683.16,3938694.066
654357,654358,2018,12,21,11,금요일,20181221,중상사고,차대사람,0,...,NaN,NaN,O,O,중앙선 침범,50,50130,50130320,14118871.68,3938561.182


In [80]:
label['total'] = label[['DEATH','SWPSN','SINJPSN','INJPSN']].sum(axis = 1)

In [81]:
label = label.drop(['X','Y'],axis = 1)

In [120]:
raw_x = label[['OCCU_MT','OCCU_DE','OCCU_TM','OCCU_DAY','OCCU_DATE','OCCU_YEAR','total','EMD_CD']]
raw_y = label[['total']]

In [121]:
raw_x['OCCU_DAY'] = raw_x['OCCU_DAY'].astype('category')

/Users/yangdongjae/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
raw_x['OCCU_DAY']

0         토요일
1         일요일
2         월요일
3         화요일
4         일요일
         ... 
654354    토요일
654355    토요일
654356    화요일
654357    금요일
654358    토요일
Name: OCCU_DAY, Length: 654359, dtype: category
Categories (7, object): [금요일, 목요일, 수요일, 월요일, 일요일, 토요일, 화요일]

In [123]:
train_x, test_x, train_y, test_y = train_test_split(raw_x, raw_y, test_size=0.2, random_state=42)

In [124]:
train_ds = lgb.Dataset(train_x, label = train_y)
test_ds = lgb.Dataset(test_x, label = test_y)

In [125]:
params = {'learning_rate': 0.001, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [126]:
model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.10846
[200]	valid_0's l2: 0.924746
[300]	valid_0's l2: 0.770026
[400]	valid_0's l2: 0.644956
[500]	valid_0's l2: 0.540178
[600]	valid_0's l2: 0.447108
[700]	valid_0's l2: 0.376598
[800]	valid_0's l2: 0.314743
[900]	valid_0's l2: 0.263049
[1000]	valid_0's l2: 0.221939
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.221939


In [127]:
predict_train = model.predict(train_x)
predict_test = model.predict(test_x)

In [128]:
model = lgb.train(params , train_ds , 1000 , test_ds , verbose_eval = 100 , early_stopping_rounds = 100)

Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 1.10846
[200]	valid_0's l2: 0.924746
[300]	valid_0's l2: 0.770026
[400]	valid_0's l2: 0.644956
[500]	valid_0's l2: 0.540178
[600]	valid_0's l2: 0.447108
[700]	valid_0's l2: 0.376598
[800]	valid_0's l2: 0.314743
[900]	valid_0's l2: 0.263049
[1000]	valid_0's l2: 0.221939
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.221939


In [ ]:
predict_train = model.predict(train_x)
predict_test = model.predict(test_x)

In [ ]:
mse = mean_squared_error(test_y, predict_test)
r2 = r2_score(test_y, predict_test)

In [ ]:
print('Mean squared error: ', mse)
print('R2 score: ', r2)

In [ ]:
final_result = pd.concat([test_y.reset_index(drop=True), pd.DataFrame(predict_test)], axis = 1)

In [ ]:
final_result.columns = ['label','predict']

In [ ]:
sns.regplot(x = 'label', y = 'predict', data = final_result)